# Relational Data 
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-benchmarks/blob/main/docs/source/notebooks/tool_usage/relational_data.ipynb)

In this task, an agent is given access to a set of tools that can be used to make queries across 3 relational tables.

The tables contain information about users, locations and foods. The agent must answer questions about the data using the provided tools.

The underlying data looks like this:

User data:

```json
{
    "id": 1,
    "name": "Alice",
    "email": "alice@gmail.com",
    "location": 1,
    "favorite_color": "red",
    "favorite_foods": [1, 2, 3],
},
{
    "id": 21,
    "name": "Bob",
    "email": "bob@hotmail.com",
    "location": 2,
    "favorite_color": "orange",
    "favorite_foods": [4, 5, 6],
}
```

Food data:

```json
{
    "id": 1,
    "name": "Pizza",
    "calories": 285,  # Calories per serving
    "allergic_ingredients": ["Gluten", "Dairy"],
},
{
    "id": 2,
    "name": "Chocolate",
    "calories": 50,  # Calories per serving
    "allergic_ingredients": ["Milk", "Soy"],
},
```

The tools allow to look up information based on ids (e.g., `get_user_email` takes a user id and returns the email),
and to search (e.g., `find_foods_by_name` takes a food name and returns a list of results.

In [5]:
from langchain_benchmarks import clone_public_dataset, registry

For this code to work, please configure LangSmith environment variables with your credentials.

In [6]:
task = registry["Tool Usage - Relational Data"]

Clone the dataset associaetd with this task

In [7]:
clone_public_dataset(task.dataset_id, dataset_name=task.name)

Dataset Tool Usage - Relational Data already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/datasets/69c0e0d0-91b5-4183-bed0-6628e76964dc.


## The Environment

Let's check the environment

In [13]:
env = task.create_environment()
env.tools[:5]

[StructuredTool(name='get_user_name', description="get_user_name(user_id: int) -> str - Get the name of the user with the given user ID.\n\n        Args:\n            user_id: The user's ID.\n\n        Returns:\n            The user's name.", args_schema=<class 'pydantic.v1.main.get_user_nameSchemaSchema'>, func=<function get_available_functions.<locals>.get_user_name at 0x7fc3d10904c0>),
 StructuredTool(name='list_user_ids', description='list_user_ids() -> List[str] - List all the user IDs.', args_schema=<class 'pydantic.v1.main.list_user_idsSchemaSchema'>, func=<function get_available_functions.<locals>.list_user_ids at 0x7fc3d1090670>),
 StructuredTool(name='find_users_by_name', description='find_users_by_name(name: str) -> List[langchain_benchmarks.tool_usage.tasks.relational_data.SearchHit] - Find users with the given name.\n\n        Args:\n            name: The name to search for.\n\n        Returns:\n            The list of matching users.', args_schema=<class 'pydantic.v1.main

In [17]:
env.tools[0].invoke({"user_id": 21})

'Bob'

In [20]:
env.tools[3].invoke({"city": "LA"})

[{'id': 2, 'city': 'Los Angeles'},
 {'id': 1, 'city': 'New York'},
 {'id': 3, 'city': 'Chicago'},
 {'id': 4, 'city': 'Houston'},
 {'id': 5, 'city': 'Miami'}]

## Define an agent

Let's build an agent that we can use for evaluation.

In [21]:
from langchain_benchmarks.tool_usage import agents

In [22]:
agent_factory = agents.OpenAIAgentFactory(task, model="gpt-3.5-turbo-16k")

Let's test that our agent works

In [23]:
agent = agent_factory()

In [24]:
agent.invoke({"question": "who is bob?"})

{'question': 'who is bob?',
 'output': 'Bob is a user with the ID 21.',
 'intermediate_steps': [(AgentActionMessageLog(tool='find_users_by_name', tool_input={'name': 'bob'}, log="\nInvoking: `find_users_by_name` with `{'name': 'bob'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "name": "bob"\n}', 'name': 'find_users_by_name'}})]),
   [{'id': 21, 'name': 'Bob'},
    {'id': 41, 'name': 'Donna'},
    {'id': 1, 'name': 'Alice'},
    {'id': 35, 'name': 'Charlie'},
    {'id': 42, 'name': 'Eve'},
    {'id': 43, 'name': 'Frank The Cat'}])]}

## Eval

Let's evaluate an agent now

In [25]:
from langsmith.client import Client

from langchain_benchmarks.tool_usage import STANDARD_AGENT_EVALUATOR

In [26]:
client = Client()

In [27]:
test_run = client.run_on_dataset(
    dataset_name=task.name,
    llm_or_chain_factory=agent_factory.create,
    evaluation=STANDARD_AGENT_EVALUATOR,
    verbose=True,
    tags=["openai-functions"],
)

View the evaluation results for project 'test-fixed-self-71' at:
https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/projects/p/1c2f10b1-370d-4062-9397-bab8189e8b95?eval=true

View all tests for Dataset Tool Usage - Relational Data at:
https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/datasets/69c0e0d0-91b5-4183-bed0-6628e76964dc
[------------------------------------------------->] 21/21
 Eval quantiles:
                                    0.25       0.5      0.75      mean  \
Intermediate steps correctness  1.000000  1.000000  1.000000  0.761905   
# steps / # expected steps      1.000000  1.000000  1.000000  0.928571   
correctness                     1.000000  1.000000  1.000000  0.809524   
execution_time                  4.253613  4.253613  4.253613  4.253613   

                                    mode  
Intermediate steps correctness  1.000000  
# steps / # expected steps      1.000000  
correctness                     1.000000  
execution_time   

# Inspect

Here, we'll take a look at the underlying results a little bit.

In [28]:
import pandas as pd

df = test_run.to_dataframe()
df = pd.json_normalize(df.to_dict(orient="records"))

df["correctness"].mean()

0.8095238095238095

In [29]:
df["num_expected_steps"] = df["reference.expected_steps"].apply(len)
df["actual_number_of_steps"] = df["output.intermediate_steps"].apply(len)

df.head()

,Intermediate steps correctness,# steps / # expected steps,correctness,execution_time,input.question,output.question,output.output,output.intermediate_steps,reference.reference,reference.order_matters,reference.expected_steps,num_expected_steps,actual_number_of_steps
0,1,1.0,1,4.253613,do bob and alice live in the same city?,do bob and alice live in the same city?,"No, Bob and Alice do not live in the same city...",[(tool='find_users_by_name' tool_input={'name'...,no,False,"[find_users_by_name, get_user_location, get_ci...",5,5
1,0,0.0,0,4.253613,Is it likely that Donna is outside with an umb...,Is it likely that Donna is outside with an umb...,"I'm sorry, but I don't have access to real-tim...",[],yes,False,"[find_users_by_name, get_user_location, get_cu...",4,0
2,1,1.0,1,4.253613,do alice and charlie use the same email provider?,do alice and charlie use the same email provider?,"No, Alice and Charlie do not use the same emai...",[(tool='find_users_by_name' tool_input={'name'...,no,True,"[find_users_by_name, get_user_email, get_user_...",3,3
3,0,0.0,0,4.253613,Is it likely that Donna is awake right now?,Is it likely that Donna is awake right now?,"I'm sorry, but I don't have access to informat...",[],yes,True,"[find_users_by_name, get_user_location, get_cu...",3,0
4,0,1.0,1,4.253613,Donna is about to go outside. Does she need an...,Donna is about to go outside. Does she need an...,Donna is at location 4 and the current weather...,[(tool='find_users_by_name' tool_input={'name'...,yes,True,"[find_users_by_name, get_user_location, get_cu...",3,3


In [32]:
df.sort_values("actual_number_of_steps", ascending=False).head()

,Intermediate steps correctness,# steps / # expected steps,correctness,execution_time,input.question,output.question,output.output,output.intermediate_steps,reference.reference,reference.order_matters,reference.expected_steps,num_expected_steps,actual_number_of_steps
0,1,1.0,1,4.253613,do bob and alice live in the same city?,do bob and alice live in the same city?,"No, Bob and Alice do not live in the same city...",[(tool='find_users_by_name' tool_input={'name'...,no,False,"[find_users_by_name, get_user_location, get_ci...",5,5
2,1,1.0,1,4.253613,do alice and charlie use the same email provider?,do alice and charlie use the same email provider?,"No, Alice and Charlie do not use the same emai...",[(tool='find_users_by_name' tool_input={'name'...,no,True,"[find_users_by_name, get_user_email, get_user_...",3,3
4,0,1.0,1,4.253613,Donna is about to go outside. Does she need an...,Donna is about to go outside. Does she need an...,Donna is at location 4 and the current weather...,[(tool='find_users_by_name' tool_input={'name'...,yes,True,"[find_users_by_name, get_user_location, get_cu...",3,3
5,0,1.0,0,4.253613,whats the name of the city where bob lives?,whats the name of the city where bob lives?,The name of the city where Bob lives is New York.,[(tool='list_user_ids' tool_input={} log='\nIn...,Los Angeles,True,"[find_users_by_name, get_user_location, get_ci...",3,3
6,1,1.0,1,4.253613,what is the current users favorite color and n...,what is the current users favorite color and n...,The current user's favorite color is yellow an...,[(tool='get_current_user_id' tool_input={} log...,yellow and Charlie,True,"[get_current_user_id, get_user_favorite_color,...",3,3
